<a href="https://colab.research.google.com/github/tantowijh/colab-collection/blob/main/Torrent_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <img src='https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png' height="45" alt="rclone"/>

In [ ]:
# ============================= FORM ============================= #
#@title <font size="4"><b>Step 1. </b>Upload your <code>rclone.conf</code> profile</font> { vertical-output: true }
#@markdown <h6>*<i>Required to mount your drive!</i></h4>
## @markdown <h7>Your <code>rclone.conf</code> file is located at:</i></h4>
# @markdown ### <font size=1px><i>For not able to upload local file : https://stackoverflow.com/a/58661947</font>
# ================================================================ #
MODE = "RCONFIG"
from os import path as _p
from IPython.display import clear_output

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/RLabClone/master/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))  # nosec

import importlib, rlab_utils
from google.colab import files  # pylint: disable=import-error #nosec
from rlab_utils import checkAvailable, runSh, PATH_RClone_Config, prepareSession


def generateUploadList():
    prepareSession()
    if checkAvailable("/content/upload.txt"):
        runSh("rm -f upload.txt")
    runSh(
        f"rclone --config {PATH_RClone_Config}/rclone.conf lsf {REMOTE}: --include '{QUERY_PATTERN}' --drive-shared-with-me --files-only --max-depth 1 > /content/upload.txt",
        shell=True,  # nosec
    )


def uploadLocalFiles():
    prepareSession()
    if MODE == "UTILS":
        filePath = "/root/.ipython/rlab_utils.py"
    elif MODE in ("RCONFIG", "RCONFIG_append"):
        filePath = f"{PATH_RClone_Config}/rclone.conf"
    else:
        pass

    try:
      if checkAvailable(filePath):
        runSh(f"rm -f {filePath}")
      print("Select file from your computer.\n")
      uploadedFile = files.upload()
      fileNameDictKeys = uploadedFile.keys()
      fileNo = len(fileNameDictKeys)
      if fileNo > 1:
          for fn in fileNameDictKeys:
              runSh(f'rm -f "/content/{fn}"')
          return print("\nPlease only upload a single config file.")
      elif fileNo == 0:
          return print("\nFile upload cancelled.")
      elif fileNo == 1:
          for fn in fileNameDictKeys:
              if checkAvailable(f"/content/{fn}"):
                  if MODE == "RCONFIG_append":
                    import urllib
                    urllib.request.urlretrieve("https://biplobsd.github.io/RLabClone/res/rclonelab/rclone.conf",
                                               "/usr/local/sessionSettings/rclone.conf")
                    with open(f"/content/{fn}", 'r+') as r:
                      new_data = r.read()
                      runSh(f'rm -f "/content/{fn}"')
                    with open(filePath, 'r+') as f:
                      old_data = f.read()
                      f.seek(0)
                      f.truncate(0)
                      f.write(old_data + new_data)
                    print("\nUpdate completed.")
                  else:
                    runSh(f'mv -f "/content/{fn}" {filePath}')
                    runSh(f"chmod 666 {filePath}")
                    runSh(f'rm -f "/content/{fn}"')
                    importlib.reload(rlab_utils)
                    print("Upload completed.")
              return
      else:
          print("\nNo file")
          return
    except:
      return print("\nUpload process Error.")


if MODE == "GENERATELIST":
    generateUploadList()
else:
    uploadLocalFiles()


In [ ]:
# ============================= FORM ============================= #
#@title <font size="4"><b>Step 2</b>. Mount your Drive with rclone</font> { vertical-output: true }
Cache_Directory = "DISK"
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
#####################################
if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from rlab_utils import (
    runSh,
    prepareSession,
    PATH_RClone_Config,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#127ACC;margin-top:0px;">
                  Choose Drive to Mount:</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))

prepareSession()
content = open(f"{PATH_RClone_Config}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {PATH_RClone_Config}/rclone.conf" \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

## Setup the Google drive permissions

In [ ]:
#@markdown Click this box, then click on the output link of this box. Sign in with google account to access your drive. Lastly enter the access code in this bar

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#@title **[1] Run this code if you want a longer run time (Optional but recommended)</font>**
#@markdown *This cell runs a JS code that automatically reconnects you to avoid inactivity due to idle.*

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function connectRefresher() {
       window.ConnectButtonIntervalId = setInterval(function ConnectButton(){
                console.log("connected"); 
                document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
                document.querySelector("colab-sessions-dialog").shadowRoot.querySelector("#footer > div > paper-button").click();
                console.log("closed the dialog!!"); 
            },60000);
    }
    
 function clearRefresher() { 
           console.log("clear Interval called !!");
           clearInterval(window.ConnectButtonIntervalId);
    }

 connectRefresher(); //to connect the refresher
 clearRefresher(); //to disconnect the refresher
'''))

clear_output()

## Start
### Enter the magnet link of torrent in the input box or put the .torrent files in tor folder of the your drive ( create the folder if it doesn't exist)


In [ ]:
#@title Torrent Class
#@markdown *Please make sure to fill the path!*
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent
!pip install termcolor
 
import libtorrent as lt
from queue import Queue
from threading import Thread
import time
import os
import datetime
from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets
from tqdm.notebook import tqdm
 
 
 
from google.colab import drive
drive.mount("/content/drive")
 
from google.colab import files
from termcolor import cprint
 

TorrentFilePath = "/content/drive/Shareddrives/05. Torrent/01. Torrent Files" #@param {type:"string"}
DownloadingPath = '/content/drive/Shareddrives/05. Torrent/02. Incomplete' #@param {type:"string"}
DownloadedPath = '/content/drive/Shareddrives/05. Torrent/03. Downloaded' #@param {type:"string"}

 
tor_path = TorrentFilePath+'/'
save_path = DownloadingPath+'/'
completed_path = DownloadedPath+'/'



for path in [TorrentFilePath, DownloadingPath, DownloadedPath]:
 if not os.path.exists(path):
	  os.makedirs(path)

# define the session of libtorrent
ses = lt.session()
ses.listen_on(6881, 6891)
# sett = lt.session_settings()
# sett = {'allow_multiple_connections_per_ip': True,
#         'dont_count_slow_torrents': True,
#         'active_downloads': 10,
#         'active_seeds': 4,
#         'active_checking': 3, }
# ses.set_settings(sett)
 
# Gui
layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
 
 
 
class Torrent(Thread):
    state_str = [
        "Queued",
        "Checking",
        "Downloading Metadata",
        "Downloading",
        "Finished",
        "Seeding",
        "Allocating",
        "Checking Fastresume",
    ]
    tor_paths = {}  # Keeps the track of which torrents are running and which are not
    tor_file_status = {}
    STORAGE_SIZE = 65
    current_torrent_size = 0
    timestamp = 0
    torrent_progress= {}
 
    def __init__(self):
        print("got here")
        Thread.__init__(self)
        print("Thread class initalized")
 
 
 
    def run(self):
        self.magnets()
 
    # Get the torrents from magnet links
    def magnets(self):
        params = {
            'save_path': save_path,
            'storage_mode': lt.storage_mode_t(2),
            # 'paused': False,
            # 'auto_managed': True,
            # 'duplicate_is_error': True, 
            'file_priorities': [0] * 1000
        }
        magnet_link= ''
        while True:
            print("\nEnter Magnet Link Or Type Exit: ")
            magnet_link = input('\n')
            if magnet_link.lower() == "exit":
                break
            # if not magent.link.lower().startswith('magnet'): continue
            # print(magnet_link)
            # print(type(ses),type(magnet_link), type(params))
            # params['uri'] = lt.add_magnet_uri(ses, magnet_link, params)
            handle = lt.add_magnet_uri(ses, magnet_link, params)
            # handle = ses.add_torrent(params)
            self.tor_file_status[handle] = False
            cprint('Magnet link added {}'.format(len(self.tor_file_status)),'green')
 
    def add_torrent(self, link):
        # print(link)
        ti = lt.torrent_info(tor_path+link)
        torrent_size = ti.total_size() / (1024 * 1024 * 1024)
 
        if torrent_size > self.STORAGE_SIZE:
            if not os.path.exists(tor_path + 'oversized_torrent/'):
                os.makedirs(tor_path + 'oversized_torrent/')
            os.replace(tor_path+link, tor_path + 'oversized_torrent/' + link.split('/')[-1])
            # try:
            #   ses.remove_torrent(torrent)
            # except:
            #   pass
            return True
 
        # If system can't accommodate more torrent, then dont add the param to the torrent
        if self.current_torrent_size + torrent_size > self.STORAGE_SIZE:
            new_timestamp = ts = datetime.datetime.now().timestamp() + (self.current_torrent_size + torrent_size - self.STORAGE_SIZE) * 100
            
            if self.timestamp == 0: 
                self.timestamp = new_timestamp
            
            self.timestamp = min(new_timestamp, self.timestamp)
            return False
 
        # else add the torrent to session
        params = {
            'save_path': save_path,
            'storage_mode': lt.storage_mode_t(2),
            # 'paused': False,
            # 'auto_managed': True,
            # 'duplicate_is_error': True,
            'ti': ti
        }
 
        try:
            self.current_torrent_size += torrent_size
            ses.async_add_torrent(params)
            print('Added ', link.split('/')[-1])
            self.current_torrent_size += torrent_size
            # print("Torrent added !! ",link)
        except RuntimeError as e:
            cprint('!! Torrent already present', 'red')
        return True
 
    def load_torrents(self):
        # get the tor files from the drive
        paths = os.listdir(tor_path)
        paths = [path for path in paths if path.endswith('.torrent')]
        # print(paths)
        # print(self.tor_paths)
 
        for path in paths:
            if path not in self.tor_paths.keys():
                self.tor_paths[path] = False
 
        # Adding torrents from drive  to the list
        for tor, status in dict(self.tor_paths).items():
            if not status:
                self.tor_paths[tor] = self.add_torrent(tor)
 
    def check(self):
        # clear_output()
        cprint('\n'+('-' * 60))
        for torrent in ses.get_torrents():
            s = torrent.status()
 
            if self.state_str[s.state] in ["Allocating", "Downloading Metadata"]:
              print(torrent.name(),'is',self.state_str[s.state])
              continue
            
            if not torrent.has_metadata():
              continue
 
            print(torrent.name())
            cprint('{:.2f}% of {:.2f}GB ( down: {:.1f} mb/s  up: {:.1f} kB/s peers: {:d}) {} \n'.format(
                s.progress * 100, torrent.get_torrent_info().total_size() / (1024 * 1024 * 1024),
                s.download_rate / 1000000,
                s.upload_rate / 1000, s.num_peers, self.state_str[s.state]), 'magenta', 'on_white',attrs=['bold'])
 
            if (torrent.file_priority(0) == 0):
                ses.remove_torrent(torrent)
                print('Metadata downloaded for ', torrent.name())
 
 
            # If torrent is seeding, then move it to new location and remove it from session
            if (torrent.is_seed()):
                # print(torrent.save_path())
 
                # In case of only metadata downloading magnetic link, remove only torrent handle
                print('Checking !!')
                torrent_size = torrent.get_torrent_info().total_size() / (1024 * 1024 * 1024) # Get the size for total size
                torrent.move_storage(completed_path)
                torrent.force_recheck()
                print("Checking Done")
 
                # while not torrent.is_seed():
                os.remove(os.path.join(tor_path, torrent.name() + ".torrent"))
                # torrent_size = torrent.total_size() / (1024 * 1024 * 1024)
                self.current_torrent_size -= torrent_size
                ses.remove_torrent(torrent)
        # while (True):
        #     fast_check()
        #     print()
        #     if len(ses.get_torrents()) == 0:
        #         exit()
        #     time.sleep(60)
 
    ## Saves the torrent as .tor files
    def save_tor_file(self, torrent):
        # print(1, torrent.name(), torrent.has_metadata())
        if not torrent.has_metadata():
            return False
 
        if os.path.exists(os.path.join(tor_path, torrent.name() + ".torrent")): return True
 
        torrent_info = torrent.get_torrent_info()
        torrent_file = lt.create_torrent(torrent_info)
        torrent_path = os.path.join(tor_path, torrent.name() + ".torrent")
        with open(torrent_path, "wb") as f:
            f.write(lt.bencode(torrent_file.generate()))
        print('tor file created: ', torrent_path, '\n')
        ses.remove_torrent(torrent)
        print([t.name() for t in ses.get_torrents()])
        return True
 
    def gui_check(self):
        # clear_output()
        # cprint('\n'+('-' * 60))
        for torrent in ses.get_torrents():
            s = torrent.status()
 
 
 
            if self.state_str[s.state] in ["Allocating", "Downloading Metadata"]:
              print(torrent.name(),'is',self.state_str[s.state])
              continue
            
            if not torrent.has_metadata():
              continue
 
            # tqdm progress bar
            if torrent in self.torrent_progress.keys():
                bar = self.torrent_progress[torrent]
                # bar.unpause()
                
 
                # bar.display(msg=torrent.name(), close=True)
                bar.write(torrent.name(), end='\n', nolock=False)
 
                bar.reset()
                bar.n = round(s.progress*torrent.get_torrent_info().total_size() / (1024 * 1024 * 1024), 2)
                bar.last_print_n = round(s.progress*torrent.get_torrent_info().total_size() / (1024 * 1024 * 1024), 2)
                bar.desc=" | ".join( [ torrent.name()[:40],
                            self.state_str[s.state], 
                            ])
                
                d = {'down':str(round(s.download_rate / 1000000, 1))+'mb/s', 'up':str(round(s.upload_rate / 1000,1))+'kb/s', 'peers':s.num_peers}
                bar.set_postfix(d)
                bar.refresh()
                # bar.pause()
            
            else:  
                self.torrent_progress[torrent] = tqdm(total=round(torrent.get_torrent_info().total_size() / (1024 * 1024 * 1024),2),
                                                      dynamic_ncols=True,
                                                      unit = 'mb',
                                                      desc=" | ".join( [ torrent.name()[:40],
                                                                  self.state_str[s.state], 
                                                                 ]),
                                                      postfix={}
 
                                                 )
 
            # print(self.torrent_progress)
            # print(torrent.name())
            # cprint('{:.2f}% of {:.2f}GB ( down: {:.1f} mb/s  up: {:.1f} kB/s peers: {:d}) {} \n'.format(
            #     s.progress * 100, torrent.get_torrent_info().total_size() / (1024 * 1024 * 1024),
            #     s.download_rate / 1000000,
            #     s.upload_rate / 1000, s.num_peers, self.state_str[s.state]), 'magenta', 'on_white',attrs=['bold'])
 
            if (torrent.file_priority(0) == 0):
                ses.remove_torrent(torrent)
                print('Metadata downloaded for ', torrent.name())
 
 
            # If torrent is seeding, then move it to new location and remove it from session
            if (torrent.is_seed()):
                # print(torrent.save_path())
 
                # In case of only metadata downloading magnetic link, remove only torrent handle
                print('Checking !!')
                torrent.move_storage(completed_path)
                torrent.force_recheck()
                print("Checking Done")
 
                if torrent.is_seed:
                    os.remove(os.path.join(tor_path, torrent.name() + ".torrent"))
                    ses.remove_torrent(torrent)
                    
 
 
    def status_check(self):
 
        while True:
            # Regular checking and status printing
            self.check()
 
            # Load awaiting torrents
            if datetime.datetime.now().timestamp() > self.timestamp:
                # print('Part 1')
                t.load_torrents()
                # print(self.tor_paths)
 
            # Download tor files for awaiting torrents
            if len(self.tor_file_status) != 0:
                # print('Part 2')
                for torrent, status in dict(self.tor_file_status).items():
                    if status:
                        del self.tor_file_status[torrent]
                    else:
                        self.tor_file_status[torrent] = self.save_tor_file(torrent)
            time.sleep(200)
            if len(ses.get_torrents()) == 0:
              exit(0)
 
t = Torrent()
t.daemon = True
t.start()
 
# display(*download_bars)
t.status_check()
# t.magnets()
 
t.join()
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-libtorrent is already the newest version (1.1.5-1build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
got here
Thread class initalized

Enter Magnet Link Or Type Exit: 
------------------------------------------------------------
Added  Strike the Blood IV.torrent
Added  [Foxy-Subs] Strike the Blood V Final - 01-0